In [ ]:
import socket
import threading
import pickle
import cv2
import numpy
import struct



In [ ]:
s = socket.socket()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(('',3333))
s.listen()

session , addr = s.accept()

In [ ]:
cap=cv2.VideoCapture(1) # 1 based on your camera
def send():    
    while True:
        ret , photo2 = cap.read() 
        image2 = pickle.dumps(photo2)        
        img_size2 = struct.pack("L", len(image2)) 
        session.sendall(img_size2 + image2)
    cv2.destroyAllWindows()
    cap.release()
    
t1=threading.Thread(target=send)
t1.start()

size_of_msg=struct.calcsize("L")
data=b''

while True:
    while len(data) < size_of_msg:
        data += session.recv(1024)
        
    true_msg_size = data[:size_of_msg]
    data = data[size_of_msg:]
    msg_size = struct.unpack("L", true_msg_size)[0]
    
    while len(data) < msg_size:
        data += session.recv(4096)
        
    image_data = data[:msg_size]
    data = data[msg_size:]
    image = pickle.loads(image_data)
    ret1 , live = cap.read()
    frame = cv2.resize(live, (240,240))
    image[240:,400:] = frame
    cv2.imshow('in receiver file of 0 camera', image)
    if cv2.waitKey(1) == 13:
        break
        
cv2.destroyAllWindows()
cap.release()
    